# Implémentez un modèle de scoring

*Notebook modélisation*

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

**Mission :**

- Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

- Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.

- Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.

- Mettre en œuvre une approche globale MLOps de bout en bout, du tracking des expérimentations à l’analyse en production du data drift.
    + Dans le notebook d’entraînement des modèles, générer à l’aide de MLFlow un tracking d'expérimentations
    + Lancer l’interface web 'UI MLFlow" d'affichage des résultats du tracking
    + Réaliser avec MLFlow un stockage centralisé des modèles dans un “model registry”
    + Tester le serving MLFlow
    + Gérer le code avec le logiciel de version Git
    + Partager le code sur Github pour assurer une intégration continue
    + Utiliser Github Actions pour le déploiement continu et automatisé du code de l’API sur le cloud
    + Concevoir des tests unitaires avec Pytest (ou Unittest) et les exécuter de manière automatisée lors du build réalisé par Github Actions

## 1 - Préparer l'environnement d'expérimentation

### 1.1 - Imports

In [3]:
import mlflow
import mlflow.tracking
from mlflow.models import infer_signature

import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("mlflow version", mlflow.__version__)
print("sklearn version", sklearn.__version__)

import warnings
warnings.filterwarnings("ignore")

numpy version 1.26.4
pandas version 2.1.4
mlflow version 2.20.1
sklearn version 1.6.1


### 1.2 - Lancement du serveur MLFlow

Dans le terminal de Powershell Prompt d'Anaconda, lancer cette instruction :

mlflow server --host 127.0.0.1 --port 5000 --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns

Cette commande permet de :
- Stocker les métadonnées du Model Registry dans une base de données locale SQLite.
- Stocker les artefacts (modèles, fichiers) en local.

### 1.3 - Initialisation du Tracking MLFlow de test

In [6]:
import os

# Ignorer Git
os.environ['GIT_PYTHON_REFRESH'] = 'quiet'

# Vérification si la variable d'environnement est bien définie
print(os.environ.get('GIT_PYTHON_REFRESH'))

quiet


In [7]:
# Définir le serveur de tracking (local ou distant) - ici local
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Démarrer une nouvelle expérimentation
mlflow.set_experiment("modele_test_quickstart_0")

2025/02/06 11:21:08 INFO mlflow.tracking.fluent: Experiment with name 'modele_test_quickstart_0' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/admin/mlruns/6', creation_time=1738837268010, experiment_id='6', last_update_time=1738837268010, lifecycle_stage='active', name='modele_test_quickstart_0', tags={}>

### 1.4 - Création d'un modèle pour tester l'initialisation

In [9]:
# Charger les données Iris (pour la démonstration)
data = load_iris()
X = data.data
y = data.target

# Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {
    "random_state": 42,
}

# Initialiser le modèle
model = LogisticRegression(**params)

# Entraînement du modèle
model.fit(X_train, y_train)
    
# Faire des prédictions sur le jeu de test
y_pred = model.predict(X_test)
#y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probabilités pour AUC binaire
y_pred_prob = model.predict_proba(X_test)  # Probabilités pour AUC

# Calculer accuracy et AUC
accuracy = accuracy_score(y_test, y_pred)
# auc = roc_auc_score(y_test, y_pred_prob) --> si le problème était binaire
auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='macro')

### 1.5 - Enregistrer le modèle et ses métadonnées dans MLflow**

In [11]:
# Démarrer une expérimentation dans MLflow
with mlflow.start_run():

    # Log the hyperparameters
    mlflow.log_params(params)
    
    # Loguer les métriques
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("auc", auc)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))
    
    # Loguer le modèle
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="modele_test_quickstart_0",
    )
    
    print(f"Accuracy: {accuracy}")
    print(f"AUC: {auc}")

# Ajouter au Model Registry
#model_version = mlflow.register_model(model_info.model_uri, "test_init_quickstart")

print(f"Modèle enregistré en version {model_info.mlflow_version}")

Successfully registered model 'modele_test_quickstart_0'.
2025/02/06 11:21:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modele_test_quickstart_0, version 1


Accuracy: 1.0
AUC: 1.0
🏃 View run traveling-hound-667 at: http://127.0.0.1:5000/#/experiments/6/runs/0134799111874d9ea53676b7c53eea3a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/6
Modèle enregistré en version 2.20.1


Created version '1' of model 'modele_test_quickstart_0'.


**Lister les versions du modèle**

In [13]:
client = mlflow.tracking.MlflowClient()
versions = client.get_latest_versions("modele_test_quickstart_0")
for v in versions:
    print(f"Version: {v.version}, Statut: {v.current_stage}")

Version: 1, Statut: None


### 1.6 - Chargez le modèle en tant que fonction Python, et l'utiliser pour une prédiction

In [15]:
from sklearn import datasets

# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


### 1.7 - Lancement de l'UI

Pour  la visualisation et la comparaison des expérimentations, ainsi que le stockage de manière centralisée des modèles.

Cliquer sur ce lien : http://127.0.0.1:5000/

### 1.8 - Initialisation du Tracking MLFlow de la mission

In [ ]:
# Définir le serveur de tracking (local ou distant) - ici local
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Démarrer une nouvelle expérimentation
mlflow.set_experiment("modele_scoring")